<a href="https://colab.research.google.com/github/yoursdream123/PIAIC-Q2-PROJECTS/blob/main/Langchain_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 3: LangChain Function/Tool Calling Calculator**
In this Project, we will create a simple LangChain Colab Notebook that uses the Google Gemini Flash model to answer user calculation questions.
## **Building a LangChain Calculator with Google Gemini Flash**
**Prerequisites**


1.   Google Colab signup.
2.   API key for Google Gemini Flash.
3.   Installed libraries: python dotenv, langchain community, google-generativeai, langchain, langchain google genai

In [20]:
!pip install python-dotenv -q
!pip install langchain-community -U -q
!pip install google-generativeai -U -q # Update `google-generativeai` package
!pip install langchain -U -q
!pip install langchain_google_genai -U -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.3 MB/s eta 0:00:00


### **Step 1: Import API keys in Colab from secrets**

In [21]:
from google.colab import userdata
userdata.get('GOOGLE_API_KEY_2')

# Step 1: Retrieve the API key from Colab's user data
api_key = userdata.get('GOOGLE_API_KEY_2')

### **Step 2: Define the Calculator Tool**
The calculator tool will handle basic arithmetic operations.

In [22]:
class Calculator:
    def calculate(self, expression: str) -> str:
        try:
            # Use Python's eval to compute the result safely
            result = eval(expression, {"__builtins__": None}, {})
            return str(result)
        except Exception as e:
            return f"Error: {e}"

### **Step 3: Create the Tool Wrapper for LangChain**
Integrate the calculator with LangChain as a tool.

In [23]:
from langchain.tools import tool

# Define the tool using a decorator
@tool
def calculator(expression: str) -> str:
    """
    Perform arithmetic calculations.
    Input: A mathematical expression as a string (e.g., "2 + 2").
    Output: Result of the calculation as a string.
    """
    calc = Calculator()
    return calc.calculate(expression)

### **Step 4: Set Up the Google Gemini Flash Model**

We initialize the Google Gemini Flash model with the API key and define the tools to be used by the agent.

In [24]:
import os
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.chains.conversation.memory import ConversationBufferMemory

# Initialize the Google Gemini Flash model
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Specify the model name, e.g., 'gemini-1.5-flash'
    api_key=userdata.get('GOOGLE_API_KEY_2'),
    temperature=0.7
)
print("Google Gemini Flash model initialized.")

Google Gemini Flash model initialized.


In [25]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import tool


# Create a list of tools
tools = [calculator]

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=gemini_model,
    agent_type="openai-function-calling"
)

# Example usage
response = agent.invoke("What is 12 * 15?")
print(response)

<ipython-input-25-c1691c3d192b>:9: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


{'input': 'What is 12 * 15?', 'output': '180'}


### **Step 5: Build the Conversational Chain**

We set up a conversational chain with memory to maintain context across multiple queries.

In [26]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Set up memory to maintain conversation context
memory = ConversationBufferMemory()

# Build the conversational chain
chain = ConversationChain(
    llm=gemini_model,
    memory=memory,
    verbose=True
)

# Example usage
response = chain.invoke("What is 12 * 15?")
print(response)

<ipython-input-26-0ebed092f483>:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
<ipython-input-26-0ebed092f483>:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  chain = ConversationChain(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is 12 * 15?
AI:

> Finished chain.
{'input': 'What is 12 * 15?', 'history': '', 'response': '12 * 15 is 180.  I arrived at that answer through a couple of mental math strategies.  I could have done standard long multiplication, but I found it quicker to break it down.  I thought of 12 as 10 + 2.  Then, 10 * 15 = 150, and 2 * 15 = 30.  Adding those together (150 + 30) gives you 180.  Of course, a calculator could also easily confirm this.\n'}


### **Step 6: Test the Calculator Tool**

We test the calculator tool by processing a series of queries and printing the responses.

In [27]:
# User query
query = "What is 15 divided by 3?"

# Get response from the system
response = chain.invoke(query)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 12 * 15?
AI: 12 * 15 is 180.  I arrived at that answer through a couple of mental math strategies.  I could have done standard long multiplication, but I found it quicker to break it down.  I thought of 12 as 10 + 2.  Then, 10 * 15 = 150, and 2 * 15 = 30.  Adding those together (150 + 30) gives you 180.  Of course, a calculator could also easily confirm this.

Human: What is 15 divided by 3?
AI:

> Finished chain.
{'input': 'What is 15 divided by 3?', 'history': 'Human: What is 12 * 15?\nAI: 12 * 15 is 180.  I arrived at that answer through a couple of mental math strategies.  I could have done standard long multiplication, but I found it quicker to brea

In [28]:
queries = [
    "What is 25 multiplied by 4?",
    "Now divide the result by 5.",
    "Add 10 to that."
]

for q in queries:
    print("Query:", q)
    print("Response:", chain.invoke(q))
    print("-" * 40)

Query: What is 25 multiplied by 4?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 12 * 15?
AI: 12 * 15 is 180.  I arrived at that answer through a couple of mental math strategies.  I could have done standard long multiplication, but I found it quicker to break it down.  I thought of 12 as 10 + 2.  Then, 10 * 15 = 150, and 2 * 15 = 30.  Adding those together (150 + 30) gives you 180.  Of course, a calculator could also easily confirm this.

Human: What is 15 divided by 3?
AI: 15 divided by 3 is 5.  This is a simple division problem.  Three goes into fifteen five times (3 x 5 = 15).  I can confirm this using a variety of computational methods, including long division, but this is easily solved through